# FOM Optimization

#### Todo:

- Check why some SDSS coordinates are not finite (or maybe their nan? not sure. check this too!).
- Determine values for collective and band fitting
- Determine values for sifto results
- Drop targets that disagree SNID vs SDSS


In [ ]:
import sys

import numpy as np
import pandas as pd
from astropy.table import Table
from pathlib import Path
from matplotlib import pyplot as plt
from sndata.csp import dr3
from sndata.sdss import sako18
from tqdm import tqdm

sys.path.insert(0, '../')
from phot_class import fom


In [ ]:
dr3.download_module_data()
sako18.download_module_data()


# SNR Distributions

In [ ]:
def get_csp_snr():
    """Return a 2d array of SNR ratios for each CSP DR3 target"""
    
    csp_snr = []
    for data_table in dr3.iter_data(verbose=True):
        snr = data_table['flux'] /  data_table['fluxerr']
        csp_snr.append(np.array(snr))
        
    return np.array(csp_snr)


In [ ]:
def get_sdss_snr(subset=['SNIa']):
    """Return a 2d array of SNR ratios for each SDSS Sako18 target
    
    Args:
        subset (list): Only consider targets having the
                       given Sako18 classifications
    """
    
    master = sako18.load_table('master')
    is_sn = np.isin(master['Classification'], subset)

    sdss_obj_ids = master['CID'][is_sn]

    sdss_snr = []
    for obj_id in tqdm(sdss_obj_ids):
        data_table = sako18.get_data_for_id(obj_id)
        snr = data_table['flux'] /  data_table['fluxerr']
        sdss_snr.append(np.array(snr))

    return np.array(sdss_snr)


In [ ]:
csp_snr = get_csp_snr()
csp_flat = np.concatenate(csp_snr)

sdss_snr = get_sdss_snr()
sdss_flat = np.concatenate(sdss_snr)
sdss_flat = sdss_flat[np.isfinite(sdss_flat)]


In [ ]:
def hist_snr_distributions(csp_snr_arr, sdss_snr_arr):
    """Plot SNR histograms for CSP and SDSS
    
    Args:
        csp_snr_arr  (ndarray): Flatt array of CSP SNR values
        sdss_snr_arr (ndarray): Flatt array of SDSS SNR values
    """
    
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(8, 4))

    ax1.hist(sdss_snr_arr, bins=np.arange(0, 100, 5), label='SDSS', density=False)
    ax2.hist(csp_snr_arr, label='CSP', color='C1', density=False)

    for survey_data, axis in zip([sdss_snr_arr, csp_snr_arr], (ax1, ax2)):
        median = np.median(survey_data)
        average = np.average(survey_data)
        axis.axvline(median, color='k', linestyle='--', label=f'median {median:.2f}')
        axis.axvline(average, color='k', linestyle=':', label=f'average {average:.2f}')

        axis.set_xlabel('SNR')
        axis.legend()


In [ ]:
hist_snr_distributions(csp_flat, sdss_flat)


In [ ]:
scale_func = np.median
scale_factor =  scale_func(sdss_flat) / scale_func(csp_flat)
scale_factor


## Scaling Classification Coordinates

In [ ]:
def get_csp_coords(results_dir):
    """Load CSP classification coordinates
    
    Args:
        results_dir (Path): Directory containing CSP fit results
    """

    # Load fitted coordinates
    csp_path = results_dir / 'csp_dr3_simple_fit_class.ecsv'
    csp_coords = Table.read(csp_path).to_pandas('obj_id')
    csp_coords['survey'] = 'CSP'

    # Load published subtypes from DR3
    csp_classes = dr3.load_table(2).to_pandas('SN')
    csp_classes = csp_classes.drop(['Dm15(tem)1', 'e_Dm15(tem)1', 'Nopt', 'NIR', 'Tspec'], axis='columns')
    csp_classes = csp_classes.rename({'Subtype1': 'type', 'Subtype2': 'Branch', 'Subtype3': 'Wang'}, axis='columns')
    
    csp_classes = csp_classes.replace('---', 'unknown')  # Objects missing subtypes
    csp_classes = csp_classes.replace('normal', 'Ia-norm')
    csp_classes = csp_classes.replace('91bg-like', 'Ia-91bg')
    csp_classes = csp_classes.replace('91T-like', 'Ia-91T')
    
    return csp_coords.join(csp_classes)


def read_peak_subtype(path):
    """Return the type summary from an SNID output file

    Args:
        path (str, Path): Path to read

    Returns:
         An astropy Table
    """

    names = ['type', 'ntemp', 'fraction', 'slope', 'redshift',
             'redshift_error', 'age', 'age_error']

    data = Table.read(
        str(path), header_start=4, data_start=4,
        data_end=28, format='ascii.basic', names=names
    ).to_pandas(index='type')
    
    sn_type, subtype, second_subtype = data.ntemp.nlargest(3).index
    assert data.loc[subtype].ntemp != second_subtype
    ntemp = data.loc[subtype].ntemp
    perc_temp = ntemp / data.loc[sn_type].ntemp

    return subtype, ntemp, perc_temp


def compile_peak_subtypes(results_dir):
    """Get peak subtypes from all output files from a previous SNID run

    Args:
        results_dir (Path): Directory of SNID outputs

    Returns:
        A DataFrame indexed by object ID
    """
    
    rows = []
    for path in results_dir.glob('*snid.output'):
        obj_id, phase, *_ = path.name.split('_')
        peak_type, ntemp, perc_temp = read_peak_subtype(path)
        rows.append([obj_id, float(phase), peak_type, ntemp, perc_temp])

    type_data = pd.DataFrame(
        rows,
        columns=['obj_id', 'phase', 'type', 'ntemp', 'perc_temp'])

    # Keep only the spectra nearest peak
    type_data['abs_phase'] = type_data.phase.abs()
    type_data = type_data.sort_values('abs_phase', ascending=True)
    type_data = type_data.drop_duplicates(keep='first', subset='obj_id')

    type_data['obj_id'] = type_data['obj_id'].astype('str')
    return type_data.set_index('obj_id')


def get_sdss_coords(results_dir, snid_dir=None):
    """Load SDSS classification coordinates
    
    Args:
        results_dir (Path): Directory containing SDSS fit results
        snid_path   (Path): Optional directory SNID typing results 
    """
    
    # Load fitted coordinates
    sdss_path = results_dir / 'sdss_sako18_simple_fit_class.ecsv'
    sdss_coords = Table.read(sdss_path).to_pandas('obj_id')
    sdss_coords['survey'] = 'SDSS'
    sdss_coords = sdss_coords.replace([np.inf, -np.inf], np.nan).dropna()

    # Load subtypes determined from SNID
    subtypes_rlap_5_path = snid_dir / 'subtype_rlap_5'
    subtypes_rlap_5 = compile_peak_subtypes(subtypes_rlap_5_path)

    subtypes_rlap_10_path = snid_dir / 'subtype_rlap_10'
    subtypes_rlap_10 = compile_peak_subtypes(subtypes_rlap_10_path)

    subtypes_rlap_5.update(subtypes_rlap_10)
    sdss_coords['type'] = subtypes_rlap_5['type']
    sdss_coords['type'] = sdss_coords['type'].fillna('unknown')
        
    return sdss_coords
    

In [ ]:
band_or_collective = 'collective'
ext_results_dir = Path('.').resolve().parent / 'results' / f'{band_or_collective}_fits' / 'with_ext'
snid_results_dir = Path('.').resolve().parent / 'results' / 'snid'
csp_coords = get_csp_coords(ext_results_dir)
sdss_coords = get_sdss_coords(ext_results_dir, snid_results_dir)

# Combine coordinates from SDSS and CSP
coordinates = pd.concat([csp_coords, sdss_coords])
coordinates = coordinates.set_index(['survey', coordinates.index])

# Make type column a combination of CSP and SNID subtypes
coordinates.head()


In [ ]:
def hist_coord_distributions(
        coord_df, 
        scale, 
        csp_bins=None, 
        sdss_bins=np.arange(-100, 100, 2.5),
        figsize=(8, 4)):
    """Plot histogram of classification coords for CSP adnd SDSS
    
    ``coord_df`` should have columns ``x`` and ``y`` and be indexible
    by survey (``CSP`` and ``SDSS``).
    
    Args:
        coord_df (dataframe): Dataframe of classification coordinates
        scale        (float): Rescale csp coordinates by the given value
        csp_bins   (ndarray): Bins to use when plotting the CSP histogram
        sdss_bins  (ndarray): Bins to use when plotting the CSP histogram
        figsize      (tuple): Size of the figure in inches
    """
    
    fig, axes = plt.subplots(1, 2, figsize=figsize)
    for coord, axis in zip('xy', axes):
        csp_vals = coord_df.loc['CSP'][coord]
        csp_median = np.median(csp_vals)
        sdss_vals = coord_df.loc['SDSS'][coord]
        sdss_median = np.median(sdss_vals)

        axis.hist(csp_vals * scale, bins=csp_bins, density=True, label='csp')
        axis.set_title(f'Scale factor: {scale * 100 :.1f}%')
        axis.legend()
        
        twin_x = axis.twinx()
        twin_x.hist(sdss_vals, density=True, histtype='step', bins=sdss_bins, label='sdss', color='C1')        
        twin_x.set_ylim(0, .01)
    
    axes[0].set_ylabel('Number of CSP targets')
    twin_x.set_ylabel('Number of SDSS targets')
    plt.tight_layout()
        

In [ ]:
hist_coord_distributions(coordinates, 1)
hist_coord_distributions(coordinates, scale_factor)


In [ ]:
def scatter_class_coordinates(coord_df, scale=1):
    """Scatter plot classification coordinates
    
    Args:
        coord_df (dataframe): Classification coordinates for CSP and SDSS
        scale        (float): Rescale CSP coordinates by the given value
    """
    
    sdss = coord_df.loc['SDSS']
    csp = coord_df.loc['CSP']
    
    fig, axis = plt.subplots()
    axis.scatter(sdss['x'], sdss['y'], s=10)
    axis.scatter(csp['x'] * scale, csp['y'] * scale, s=5)
    
    x_label = r'$\chi^2_{blue}$ (Ia) - $\chi^2_{blue}$ (91bg)'
    y_label = r'$\chi^2_{red}$ (Ia) - $\chi^2_{red}$ (91bg)'
    axis.set_xlabel(x_label)
    axis.set_ylabel(y_label)
    axis.set_title(f'Scale factor: {scale * 100 :.1f}%')


In [ ]:
scatter_class_coordinates(coordinates)
plt.xlim(-1500, 1000)
plt.ylim(-3000, 2000)

scatter_class_coordinates(coordinates)
plt.xlim(-100, 100)
plt.ylim(-100, 100)


In [ ]:
scatter_class_coordinates(coordinates, scale_factor)
plt.xlim(-1500, 1000)
plt.ylim(-3000, 2000)

scatter_class_coordinates(coordinates, scale_factor)
plt.xlim(-100, 100)
plt.ylim(-100, 100)


## FOM 

In [ ]:
from scipy.optimize import minimize

min_method = 'Powell'
fom_class = 'Ia-91bg'
x0 = [0, 0]

fom_coords = coordinates[coordinates.type != 'unknown']
def inv_fom(args, scale):
    xcut, ycut = args
    return 1 / fom.rectangular(
        fom_coords.type, 
        fom_coords.x * scale, 
        fom_coords.y * scale, 
        xcut, 
        ycut,
        fom_class)

min_res = minimize(
    fun=inv_fom, 
    x0=x0, 
    method=min_method,
    args=scale_factor,
    bounds=[
        (0, max(fom_coords.x)), 
        (0, max(fom_coords.y))
    ])

print('FOM =', 1 / min_res.fun)
min_res


In [ ]:
def scatter_fom(coord_df, x_cut=0, y_cut=0, scale=1, fom=None, figsize=(8, 8)):
    """Scatter plot classification coordinates and include FOM results
    
    Args:
        coord_df (dataframe): Classification coordinates for CSP and SDSS
        x_cut        (float): x cutoff used in FOM calculation
        y_cut        (float): y cutoff used in FOM calculation
        scale        (float): Rescale CSP coordinates by the given value
        figsize      (tuple): Size of the figure in inches
    """
    
    fig, axis = plt.subplots(figsize=figsize)
    for sn_type, dataframe in coord_df.groupby('type'): 
        if sn_type == 'unknown':
            axis.scatter(dataframe.x, dataframe.y, color='grey', alpha=.2, zorder=-1)
        
        else:    
            label = f'{sn_type} ({len(dataframe)})'
            axis.scatter(dataframe.x * scale, dataframe.y * scale, label=label, s=20)

    line_kwargs = dict(linestyle='--', color='black', alpha=.5)
    axis.axvline(x_cut, **line_kwargs)
    axis.axhline(y_cut, **line_kwargs, label=f'FOM = {fom: .2f}' if fom else fom)
        
    x_label = r'$\chi^2_{blue}$ (Ia) - $\chi^2_{blue}$ (91bg)'
    y_label = r'$\chi^2_{red}$ (Ia) - $\chi^2_{red}$ (91bg)'
    axis.set_xlabel(x_label)
    axis.set_ylabel(y_label)
    
    plt.legend()
    

In [ ]:
scatter_fom(
    coord_df=coordinates, 
    x_cut=min_res.x[0], 
    y_cut=min_res.x[1],
    scale=scale_factor, 
    fom=1 / min_res.fun)

plt.xlim(-100, 100)
plt.ylim(-100, 100)


In [ ]:
import itertools

from bokeh.models import Range1d
from bokeh.plotting import figure, show
from bokeh.palettes import Dark2_5 as palette
from bokeh.io import  output_notebook

output_notebook()


In [ ]:
def bokeh_fom(coord_df, x_cut=0, y_cut=0, scale=1, fom=None, figsize=(8, 8)):
    """Scatter plot classification coordinates and include FOM results
    
    Args:
        coord_df (dataframe): Classification coordinates for CSP and SDSS
        x_cut        (float): x cutoff used in FOM calculation
        y_cut        (float): y cutoff used in FOM calculation
        scale        (float): Rescale CSP coordinates by the given value
        figsize      (tuple): Size of the figure in inches
    """
    
    colors = itertools.cycle(palette) 
    fig = figure(plot_width=800, plot_height=800)
    
    
    for sn_type, dataframe in coord_df.groupby('type'): 
        if sn_type == 'unknown':
            continue
        
        fig.circle(dataframe.x * scale, dataframe.y * scale, color=next(colors))

    fig.ray(x=[y_cut], y=[y_cut], length=0, angle=0, line_width=1, color='black')
    fig.ray(x=[y_cut], y=[y_cut], length=0, angle=np.pi, line_width=1, color='black')
    
    fig.ray(x=[x_cut], y=[x_cut], length=0, angle=-np.pi / 2, line_width=1, color='black')
    fig.ray(x=[x_cut], y=[x_cut], length=0, angle=np.pi / 2, line_width=1, color='black')

    fig.xaxis.axis_label = r'$\chi^2_{blue}$ (Ia) - $\chi^2_{blue}$ (91bg)'
    fig.yaxis.axis_label = r'$\chi^2_{red}$ (Ia) - $\chi^2_{red}$ (91bg)'
    fig.x_range = Range1d(-100, 100)
    fig.y_range = Range1d(-100, 100)
    show(fig)


In [ ]:
bokeh_fom(
    coord_df=coordinates, 
    x_cut=min_res.x[0], 
    y_cut=min_res.x[1],
    scale=scale_factor, 
    fom=1 / min_res.fun)
